In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [2]:
dataset = pd.read_csv("../input/vehicle-dataset-from-cardekho/Car details v3.csv")
dataset.head()
dataset.info()

In [3]:
# first need to convert to dataframe then drop multiple columns
df = pd.DataFrame(dataset)
df.drop(df.columns[[0,1,5,6,7,9,11]], axis = 1, inplace = True)
df
df.info()

In [4]:
df.drop(df.columns[[2, 5]], axis = 1, inplace = True)
df.info()

In [5]:
#to separate int and string inside columns
df[['mileage','str']] = df['mileage'].str.split(' ', 1, expand=True)
df.info()

In [6]:
#to separate int and string inside columns
df[['max_power','bhp']] = df['max_power'].str.split(' ', 1, expand=True)
df.info()

In [7]:
df.drop(df.columns[[4, 5]], axis = 1, inplace = True)
df.info()

In [8]:
df[['mileage','max_power']] = df[['mileage','max_power']].apply(pd.to_numeric)
df.info()
df

In [24]:
DF = pd.DataFrame(df).to_numpy()
DF

In [25]:
inputs = DF[:, [1,2,3]]
outputs = DF[:,[1]]
inputs
outputs

In [26]:
inputs  = torch.from_numpy(inputs)
targets = torch.from_numpy(outputs)
inputs
targets

In [27]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

In [28]:
batch_size = 16 #8128 / 16 = 508
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [29]:
cont = 1
for xb, yb in train_dl:
    print(xb)
    print(yb)
    cont = cont + 1
    break

In [30]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

In [31]:
list(model.parameters())
preds = model(inputs.float())
preds

In [32]:
# Define loss function
loss_fn = F.mse_loss

In [55]:
targets = targets.expand(-1, 2)
targets.shape

In [57]:
loss = loss_fn(model(inputs.float()), targets)
print(loss)

In [58]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [59]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if(epoch+1)%10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [60]:
fit(100, model, loss_fn, opt, train_dl)